In [33]:
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost
import shap

In [51]:
class OurModel(layers.Layer):
    def __init__(self, num_layers):
        super(OurModel, self).__init__()

        self.fnnS = keras.layers.Dense(32)
        self.fnn = [keras.layers.Dense(32,activation='relu') for i in range(num_layers)]
        self.final = keras.layers.Dense(1)
        self.num_layers = num_layers

    def call(self, inputs):
        # inputs.shape = [batch_size, n_features]

        x = self.fnnS(inputs)
        for ii in range(self.num_layers):
            shortcut = x
            x = self.fnn[ii](x)
            #x = shortcut + x
        out = self.final(x)
        return tf.nn.tanh(out)

def main(xtrain, ytrain, xtest, ytest,num_layers,epochs,batchsz):

    input = keras.layers.Input(shape=(23), dtype=tf.float32)
    model = OurModel(num_layers) #创建你的GRU模型 初始化你的模型
    out = model(input)
    model = keras.Model(inputs=input,outputs = out)
    model.compile(optimizer = keras.optimizers.Adam(0.001),
                  loss = keras.losses.mean_absolute_error
                  ) #装载你的模型参数，设定损失函数和优化器
    model.fit(x=xtrain,y=ytrain, batch_size= batchsz,epochs=epochs,validation_data=(xtest,ytest))#训练
    model.evaluate(x=xtest,y=ytest,batch_size= batchsz) #评价训练的精度
    ypre = model.predict(x=xtest,batch_size=batchsz)
    ytrainpre = model.predict(x=xtrain,batch_size=batchsz)
    return ypre ,ytrainpre

In [52]:
rider_df = pd.read_csv('poi_bike_distance_features.csv')
rider_df = rider_df.sample(frac=0.1)

In [53]:
rider_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 894773 entries, 8016356 to 3338104
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               894773 non-null  int64  
 1   started_at               894773 non-null  object 
 2   start_station_id         894773 non-null  int64  
 3   start_lat                894773 non-null  float64
 4   start_lng                894773 non-null  float64
 5   rider_demand             894773 non-null  int64  
 6   bus_dist                 894773 non-null  float64
 7   subway_dist              894773 non-null  float64
 8   Population_density       894773 non-null  float64
 9   Residential              894773 non-null  float64
 10  Health Services          894773 non-null  float64
 11  Public Safety            894773 non-null  float64
 12  Water                    894773 non-null  float64
 13  Miscellaneous            894773 non-null  float64
 1

In [55]:
len(rider_df)

894773

In [56]:
features = rider_df.loc[:, ['start_lat', 'start_lng', 'hour', 'dayofyear', 'dayofweek', 'month', 'year',
                            'bus_dist', 'subway_dist',
                            'Population_density', 'Residential', 'Health Services', 'Public Safety', 'Water',
                            'Miscellaneous', 'Education Facility', 'Cultural Facility',
                            'Recreational Facility', 'Social Services', 'Transportation Facility',
                            'Commercial', 'Government Facility', 'Religious Institution']]

target = rider_df['rider_demand']
features['month'] = features['month'].astype('int')
s1 = MinMaxScaler()
s2 = MinMaxScaler()
features = s1.fit_transform(features)
target = s2.fit_transform(np.array(target).reshape(894773,1)).flatten()
X_train, X_test, y_train, y_test = train_test_split(features, target, train_size=0.7, random_state=1)


In [65]:
num_layers = 6
epochs = 100
batchsz = 10

In [66]:
predictions_test, predictions_train = main(X_train, y_train, X_test, y_test, num_layers,epochs,batchsz)


Epoch 1/100
62635/62635 [==============================] - 47s 741us/step - loss: 32159658.0000 - val_loss: 32061156.0000
Epoch 2/100
35668/62635 [================>.............] - ETA: 21s - loss: 32167938.0000

KeyboardInterrupt: 

In [ ]:
predictions_train = s2.inverse_transform(predictions_train)
predictions_test = s2.inverse_transform(predictions_test)
y_train = s2.inverse_transform(np.array(y_train).reshape(y_train.shape[0],1))
y_test = s2.inverse_transform(np.array(y_test).reshape(y_test.shape[0],1))
mae_train = mean_absolute_error(y_train, predictions_train)
mae_test = mean_absolute_error(y_test, predictions_test)
r2_train = r2_score(y_train, predictions_train)
r2_test = r2_score(y_test, predictions_test)
print(f"Train MAE for decision tree:{mae_train}")
print(f"Test MAE for decision tree:{mae_test}")
print(f"Train r2 score for decision tree:{r2_train}")
print(f"Test r2 score for decision tree:{r2_test}")

Train MAE for decision tree:15428712.124618215
Test MAE for decision tree:15452179.769731252
Train r2 score for decision tree:0.5917469360587939
Test r2 score for decision tree:0.5811526889785137
